# EDA precipitación de Azure Table Storage


### Librerías


In [2]:
from data_service import set_table_service, get_dataframe_from_table_storage_table
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
connection_string = os.getenv("AZ_CONNECTION_STRING")

In [5]:
table_name_represa = "InventarioRepresasColombia"
filterQuery = "PartitionKey ne 'random'"

In [6]:
table_name_represa = set_table_service(connection_string, table_name_represa)
df_represas = get_dataframe_from_table_storage_table(table_name_represa, filterQuery)
print(f"Shape: {df_represas.shape}")

Shape: (48, 12)


In [7]:
conteo_por_departamento = (
    df_represas.groupby("Departamento").size().reset_index(name="Cantidad_Represas")
)
conteo_por_departamento = conteo_por_departamento.sort_values(
    by="Cantidad_Represas", ascending=False
)

indice_a_cambiar = 0
nuevo_nombre = "Cundinamarca"
indice_a_cambiar1 = 1
nuevo_nombre1 = "Santander"

# Utiliza el método at para cambiar el nombre del departamento
conteo_por_departamento.at[indice_a_cambiar, "Departamento"] = nuevo_nombre
conteo_por_departamento.at[indice_a_cambiar1, "Departamento"] = nuevo_nombre1

# Imprime el DataFrame resultante
conteo_por_departamento.info()
conteo_por_departamento

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 2 to 6
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Departamento       10 non-null     object
 1   Cantidad_Represas  10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 540.0+ bytes


,Departamento,Cantidad_Represas
2,Antioquia,24
9,Valle del Cauca,11
7,Huila,4
1,Santander,2
8,Tolima,2
0,Cundinamarca,1
3,Boyaca,1
4,Caldas,1
5,Cauca,1
6,Cordoba,1


In [8]:
df_estandarizado = conteo_por_departamento.copy()

# Estandarizar el nombre del departamento
df_estandarizado["Departamento"] = (
    df_estandarizado["Departamento"]
    .str.lower()  # Convertir a minúsculas
    .str.replace("d.c.", "", regex=False)  # Retirar "D.C."
    .str.replace("[^\w\s]", "", regex=True)  # Retirar comas, puntos
    .str.replace("[áäâà]", "a", regex=True)  # Reemplazar tildes
    .str.replace("[éëêè]", "e", regex=True)
    .str.replace("[ñ]", "n", regex=True)
    .str.replace("[íïîì]", "i", regex=True)
    .str.replace("[óöôò]", "o", regex=True)
    .str.replace("[úüûù]", "u", regex=True)
    .str.strip()  # Retirar espacios al principio y al final
)

# Verificar el DataFrame estandarizado
print(df_estandarizado)

      Departamento  Cantidad_Represas
2        antioquia                 24
9  valle del cauca                 11
7            huila                  4
1        santander                  2
8           tolima                  2
0     cundinamarca                  1
3           boyaca                  1
4           caldas                  1
5            cauca                  1
6          cordoba                  1


In [9]:
df_represas_final = df_estandarizado.copy()
# Agregar el prefijo 'precipitacion_' a los nombres de las columnas
df_represas_final.columns = ["represas_" + col for col in df_represas_final.columns]

df_represas_final

,represas_Departamento,represas_Cantidad_Represas
2,antioquia,24
9,valle del cauca,11
7,huila,4
1,santander,2
8,tolima,2
0,cundinamarca,1
3,boyaca,1
4,caldas,1
5,cauca,1
6,cordoba,1


In [10]:
df_represas_final.to_csv("processed_tables/represas.csv", index=False)

In [22]:
total_represas_antioquia = df_represas_final[
    df_represas_final["represas_Departamento"] == "antioquia"
]["represas_Cantidad_Represas"].values[0]
total_represas = df_represas_final["represas_Cantidad_Represas"].sum()
print(
    f"Antioquia representa el {(total_represas_antioquia / total_represas)*100}% del total de represas"
)

Antioquia representa el 50.0% del total de represas
